<a href="https://colab.research.google.com/github/Kim-Yeon-Jun/NLP/blob/main/Ngram_LM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data

In [1]:
corpus = [
    "오늘 날씨 어때",
    "오늘 날씨 어때",
    "오늘 날씨 어때",
    "오늘 축구 경기",
    "오늘 경기 결과",
    "오늘 경기 결과",
    "내일 날씨",
    "내일 축구 경기",
    "내일 축구 경기",
    "축구 일정",
]

# Tokenization

In [2]:
# set() : 중복 없는 집합
vocabs = set()
for sample in corpus:
  for token in sample.split(): # 각 문장들에서 단어만을 추출
    vocabs.add(token)

In [3]:
vocabs = list(vocabs)
print(vocabs)

['날씨', '경기', '오늘', '어때', '축구', '내일', '결과', '일정']


In [4]:
idx_to_token = vocabs
#enumerate : 리스트의 idx, value를 반환
#token : idx => 받은 값을 딕셔너리의 형태로 저장
token_to_idx = {token: idx for idx, token in enumerate(idx_to_token)}
print(token_to_idx)

{'날씨': 0, '경기': 1, '오늘': 2, '어때': 3, '축구': 4, '내일': 5, '결과': 6, '일정': 7}


# Bi-gram LM

In [6]:
prev_token_to_next_token_cnt_table = {}
total_cnt = 0
for sample in corpus:
  # 반복문을 통해 각 문장별로 아래의 처리를 실행
  # 이번 문장의 단어들을 tokens에 저장
  tokens = sample.split()
  #print(tokens)

  # zip : 두개의 요소를 튜플로 묶은 후 이를 리스트로 반환
  # tokens[:-1], tokens[1:] : (처음부터 마지막 전, 처음이후부터 마지막까지) = n번 토큰, n+1번 토큰
  for prev_token, next_token in zip(tokens[:-1], tokens[1:]):
    # prev_token이 테이블에 없으면 key=prev_token 으로 {} 생성
    if prev_token not in prev_token_to_next_token_cnt_table:
      prev_token_to_next_token_cnt_table[prev_token] = {}
    # next_token이 테이블의 prev_token에 추가되어 있지 않으면 next_token을 요소에 추가하고 0으로 초기화
    if next_token not in prev_token_to_next_token_cnt_table[prev_token]:
      prev_token_to_next_token_cnt_table[prev_token][next_token] = 0
    # prev, next로 등장하는 횟수를 카운트
    prev_token_to_next_token_cnt_table[prev_token][next_token] += 1
    total_cnt += 1

In [7]:
prev_token_to_next_token_cnt_table

{'오늘': {'날씨': 3, '축구': 1, '경기': 2},
 '날씨': {'어때': 3},
 '축구': {'경기': 3, '일정': 1},
 '경기': {'결과': 2},
 '내일': {'날씨': 1, '축구': 2}}

In [8]:
import pandas as pd
# .transpose : 행과 열 바꾸기
prev_token_to_next_token_cnt_table_df = pd.DataFrame(prev_token_to_next_token_cnt_table).transpose()
prev_token_to_next_token_cnt_table_df.fillna(0, inplace=True)

In [9]:
prev_token_to_next_token_cnt_table_df

,날씨,축구,경기,어때,일정,결과
오늘,3.0,1.0,2.0,0.0,0.0,0.0
날씨,0.0,0.0,0.0,3.0,0.0,0.0
축구,0.0,0.0,3.0,0.0,1.0,0.0
경기,0.0,0.0,0.0,0.0,0.0,2.0
내일,1.0,2.0,0.0,0.0,0.0,0.0


In [10]:
prev_token_to_next_token_cnt_table_df.loc['날씨', '어때']

3.0

# P(prev_token, next_token) joint porb distribution

In [11]:
prev_token_to_next_token_cnt_table_df = prev_token_to_next_token_cnt_table_df / total_cnt

In [12]:
prev_token_to_next_token_cnt_table_df

,날씨,축구,경기,어때,일정,결과
오늘,0.166667,0.055556,0.111111,0.000000,0.000000,0.000000
날씨,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000
축구,0.000000,0.000000,0.166667,0.000000,0.055556,0.000000
경기,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111
내일,0.055556,0.111111,0.000000,0.000000,0.000000,0.000000


# P(next_token|prev_token) Conditional prob distribution

In [16]:
marginal_prob = prev_token_to_next_token_cnt_table_df.sum(axis=1)

In [17]:
marginal_prob

오늘    0.333333
날씨    0.166667
축구    0.222222
경기    0.111111
내일    0.166667
dtype: float64

P(next_token|prev_token) = P(prev_token, next_token) / P(prev_token)

In [18]:
prev_token_to_next_token_cnt_table_df = prev_token_to_next_token_cnt_table_df.copy()

In [20]:
for row_name, row in prev_token_to_next_token_cnt_table_df.iterrows():
  for col_name, val in row.items():
    prev_token_to_next_token_cnt_table_df.loc[row_name, col_name] /= marginal_prob[row_name]

In [21]:
prev_token_to_next_token_cnt_table_df

,날씨,축구,경기,어때,일정,결과
오늘,0.500000,0.166667,0.333333,0.0,0.00,0.0
날씨,0.000000,0.000000,0.000000,1.0,0.00,0.0
축구,0.000000,0.000000,0.750000,0.0,0.25,0.0
경기,0.000000,0.000000,0.000000,0.0,0.00,1.0
내일,0.333333,0.666667,0.000000,0.0,0.00,0.0


In [22]:
# P(next_token = 날씨 | prev_token = 오늘)
prev_token_to_next_token_cnt_table_df.loc['오늘', '날씨']

0.5

In [23]:
# P(next_token = 경기 | prev_token = 결과)
prev_token_to_next_token_cnt_table_df.loc['경기', '결과']

1.0

# Bi-gram을 이용한 Naive한 자동 완성 기능


In [25]:
# 마지막 단어만을 이용한 추측
x = '오늘'

input_tokens = x.split()

if input_tokens:
  last_token = input_tokens[-1]

#lookup cond prob distribution
prev_token_to_next_token_cnt_table_df.loc[last_token]

날씨    0.500000
축구    0.166667
경기    0.333333
어때    0.000000
일정    0.000000
결과    0.000000
Name: 오늘, dtype: float64

In [30]:
# get top candidates
top_k = 2
next_token_cadidates = prev_token_to_next_token_cnt_table_df.loc[last_token].sort_values(ascending = False)[:top_k]

In [31]:
next_token_candidates = next_token_cadidates.keys().tolist()
next_token_cadidates

날씨    0.500000
경기    0.333333
Name: 오늘, dtype: float64

In [37]:
def suggest(query, depth = 2, top_k=2):
  input_tokens = query.split()

  if input_tokens:
    last_token = input_tokens[-1]
  else:
    last_token = marginal_prob.sort_values(ascending=False).index[0]
  suggested_list = []
  def dfs(last_token, prev_token_list, depth):
    if depth <= 0:
      return

    # recursion
    next_token_candidates = prev_token_to_next_token_cnt_table_df.loc[last_token].sort_values(ascending=False)
    next_token_candidates = next_token_candidates[next_token_candidates > 0] # 확률이 0 이상인 것만
    last_token_list = next_token_candidates.keys().tolist()[:top_k]

    for last_token in last_token_list:
      new_suggest = prev_token_list + [last_token]
      suggested_list.append(new_suggest)

      if last_token in prev_token_to_next_token_cnt_table_df.index:
        dfs(last_token, prev_token_list = new_suggest, depth = depth-1)
  dfs(last_token, [last_token], depth)

  #concate tokens
  suggested_list = [ " ".join(tokens) for tokens in suggested_list]
  return suggested_list

In [39]:
suggest("축구",depth=2)

['축구 경기', '축구 경기 결과', '축구 일정']